In [46]:
import pygame
import math
pygame.init()
width, height = 1500, 1000
screen = pygame.display.set_mode((width, height), pygame.RESIZABLE)
pygame.display.set_caption("Lunina uchit")

#background
background_heaven = pygame.image.load("day.jpg")
background_heaven = pygame.transform.scale(background_heaven, (width, height))
background_hell = pygame.image.load("night.jpg")
background_hell = pygame.transform.scale(background_hell, (width, height))
background=background_heaven
hell = False

background_speed = 0
background_x = 0
#subject
lunina_image = pygame.image.load("lunina.png").convert()
lunina_image.set_colorkey((255,255,255))
lunina_image = pygame.transform.scale(lunina_image, (width//10, (height*3)//20))
death=0
#bullet
ball_image = pygame.image.load("potion.png").convert()
ball_image.set_colorkey((255,255,255))
ball_image = pygame.transform.scale(ball_image, (width//20, (height*3)//40))

#aim
aim_image = pygame.image.load("chert.jpg").convert()
aim_image.set_colorkey((255,255,255))
aim_image = pygame.transform.scale(aim_image, (width//12, (height*3)//20))
death_flag = False



#music
pygame.mixer.init()
pygame.mixer.music.load("bg_music.mp3")
pygame.mixer.music.play(-1) 
pygame.mixer.music.set_volume(0.25)
score_sound = pygame.mixer.Sound("helicopter.wav")
bad_sound = pygame.mixer.Sound("explos.wav")
paused = False

#start_pos
start_x, start_y = width//6, height
x, y = start_x, start_y*0.75
lunina_x, lunina_y = start_x, start_y-height//4
radius = width//38
#delta
vx, vy = 0, 0
gravity = 0.5
power = 12
angle = 45
bounce_loss = (height*7)//4000
on_ground = True
#aim
hoop1_x, hoop1_y = (width*5)//6, height//2
start_hoop1_x, start_hoop1_y = (width*5)//6, height//2
hoop2_x, hoop2_y = (width*7)//12, height//2
start_hoop2_x, start_hoop2_y = (width*7)//12, height//2
hoop_width, hoop_height = width//10, (height*3)//40
delta1=width//100
delta2=width//100

font = pygame.font.Font(None, 24)

scored_flag = False
score = 0


#functions
def rect(x,y,width,height):
    return pygame.Rect(x,y,width,height)


running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                angle = min(angle + 5, 80)
            elif event.key == pygame.K_DOWN:
                angle = max(angle - 5, 10)
            elif event.key == pygame.K_w:
                power = min(power + 1, 25)
            elif event.key == pygame.K_s:
                power = max(power - 1, 5)
            elif event.key == pygame.K_f:
                if not hell:
                    background = background_hell
                    hell = True
                else:
                    background = background_heaven
                    hell = False
            elif event.key == pygame.K_SPACE and on_ground:
                rad = math.radians(angle)
                vx = power * math.cos(rad)*width//600
                vy = -power * math.sin(rad)*height//400
                on_ground = False
            elif event.key == pygame.K_p:  
                if paused:
                    pygame.mixer.music.unpause()
                    paused = False
                else:
                    pygame.mixer.music.pause()
                    paused = True
            elif event.key == pygame.K_a:
                background_speed = (-1)*width//120
            elif event.key == pygame.K_d:
                background_speed = width//120
            elif event.key == pygame.K_r:
                death_flag = False
                score = 0
        elif event.type == pygame.VIDEORESIZE:
            width, height = event.w, event.h
            screen = pygame.display.set_mode((width, height), pygame.RESIZABLE)
            #background
            background_heaven = pygame.image.load("day.jpg")
            background_heaven = pygame.transform.scale(background_heaven, (width, height))
            background_hell = pygame.image.load("night.jpg")
            background_hell = pygame.transform.scale(background_hell, (width, height))
            background=background_heaven
            hell = False

            background_speed = 0

            #subject
            lunina_image = pygame.image.load("lunina.png").convert()
            lunina_image.set_colorkey((255,255,255))
            lunina_image = pygame.transform.scale(lunina_image, (width//10, (height*3)//20))
            death=0
            #bullet
            ball_image = pygame.image.load("potion.png").convert()
            ball_image.set_colorkey((255,255,255))
            ball_image = pygame.transform.scale(ball_image, (width//20, (height*3)//40))

            #aim
            aim_image = pygame.image.load("chert.jpg").convert()
            aim_image.set_colorkey((255,255,255))
            aim_image = pygame.transform.scale(aim_image, (width//12, (height*3)//20))
            death_flag = False


            #start_pos
            start_x, start_y = width//6, height
            x, y = start_x, start_y*0.75
            lunina_x, lunina_y = start_x, start_y-height//4
            radius = width//38
            #delta
            vx, vy = 0, 0
            gravity = height//800
            power = 12
            angle = 45
            bounce_loss = (height*7)//4000
            on_ground = True
            #aim
            hoop1_x, hoop1_y = (width*5)//6, height//2
            start_hoop1_x, start_hoop1_y = (width*5)//6, height//2
            hoop2_x, hoop2_y = (width*7)//12, height//2
            start_hoop2_x, start_hoop2_y = (width*7)//12, height//2
            hoop_width, hoop_height = width//10, (height*3)//40
            delta1=width//100
            delta2=width//100
        elif event.type == pygame.KEYUP:
            if event.key in (pygame.K_a, pygame.K_d):
                background_speed = 0
    
    if not on_ground and not paused:
        vy += gravity
        x += vx
        y += vy

    if y + radius >= height:
        y = height - radius
        vy = -vy * bounce_loss
        vx *= bounce_loss
        if abs(vy) < 1 and abs(vx) < 1:
            x, y = start_x, start_y*0.75
            vx, vy = 0, 0
            on_ground = True
            scored_flag = False

    hoop1_rect = rect(hoop1_x-background_x, hoop1_y, hoop_width, hoop_height)
    hoop2_rect = rect(hoop2_x-background_x, hoop2_y, hoop_width, hoop_height)
    ball_rect = rect(x - radius, y - radius, radius * 2, radius * 2)
    lunina_rect = rect(lunina_x-width//21, lunina_y-height//9, lunina_image.get_width(), lunina_image.get_height())

    if (ball_rect.colliderect(hoop1_rect) or ball_rect.colliderect(hoop2_rect)) and not scored_flag and not on_ground:
        scored_flag = True
        score += 1
        score_sound.play()

    if (lunina_rect.colliderect(hoop1_rect) or lunina_rect.colliderect(hoop2_rect)) and not scored_flag:
        death_flag = True
        bad_sound.play()
        hoop1_x, hoop1_y = start_hoop1_x, start_hoop1_y
        background_x = 0
        score = 0

    background_x += background_speed + width//600

    # Смещение фона относительно лягушки
    bg_x = -background_x % width

    
    screen.blit(background, (bg_x, 0))
    screen.blit(background, (bg_x - width, 0))

    if on_ground and not paused:
        points = []
        rad = math.radians(angle)
        temp_vx = power * math.cos(rad)*width//600
        temp_vy = -power * math.sin(rad)*height//400
        temp_x, temp_y = start_x, start_y*0.75
        for i in range(60):
            temp_vy += gravity
            temp_x += temp_vx
            temp_y += temp_vy
            if temp_y + radius >= height:
                break
            points.append((int(temp_x), int(temp_y)))
        if len(points) > 1:
            pygame.draw.lines(screen, (0, 0, 0), False, points, 2)



    if not paused:
        if not scored_flag:
            if hoop1_y + hoop_height >= height or hoop1_y - hoop_height <=(-1)*(height//20):
                delta1=delta1*(-1)    
            hoop1_y+=delta1
        else:
            hoop1_x, hoop1_y = start_hoop1_x, start_hoop1_y
    
        if not scored_flag:
            if hoop2_y + hoop_height >= height or hoop2_y - hoop_height <=(-1)*(height//20):
                delta2=delta2*(-1)    
            hoop2_y-=delta2
        else:
            hoop2_x, hoop2_y = start_hoop2_x, start_hoop2_y



    screen.blit(aim_image, (hoop1_x-background_x, hoop1_y))
    screen.blit(aim_image, (hoop2_x-background_x, hoop2_y))


    screen.blit(lunina_image, (lunina_x-width//21, lunina_y-height//9))
    screen.blit(ball_image, (int(x - radius), int(y - radius)))

    text_angle = font.render(f"Угол: {angle}°", True, (255, 0, 0))
    text_power = font.render(f"Сила: {power}", True, (255, 0, 0))
    text_score = font.render(f"Счёт: {score}", True, (255, 0, 0))
    screen.blit(text_angle, (width//60, height//40))
    screen.blit(text_power, (width//60, (height//40)*3))
    screen.blit(text_score, (width//60, (height//40)*5))

    if scored_flag:
        hit_text = font.render("ПОПАДАНИЕ!", True, (0, 0, 10))
        screen.blit(hit_text, ((width*5)//12, (height//40)*5))
    if death_flag:
        hit_text = font.render("LOSER!!!!!!!", True, (255, 0, 10))
        screen.blit(hit_text, ((width*5)//12, (height//40)*5))
    if score == 3:
        hit_text = font.render("OTL10", True, (255, 0, 10))
        screen.blit(hit_text, ((width*19)//40, (height//20)*5))
    if abs(background_x)>=width:
        pygame.quit()

    pygame.display.flip()
    pygame.time.delay(20)

pygame.quit()

error: video system not initialized